In [1]:
# !pip install datasets
# !pip install evaluate
# !pip install transformers[torch]
# !pip install accelerate -U

In [2]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, Dataset, DatasetDict
import torch

In [3]:
torch.cuda.is_available()

True

**Load Data and pre-process**

In [4]:
df = pd.read_json("drive/MyDrive/Data/corpus.jsonl", lines=True)
df

,argument_id,argument,stance,topic,demographic_profile
0,201900,Das Schweizer Volk hat die MEI angenommen und ...,FAVOR,Immigration,"{'gender': 'Männlich', 'age': '18-34', 'reside..."
1,201901,Eine Legalisierung von Cannabis entlasten die ...,FAVOR,Society,"{'gender': 'Männlich', 'age': '18-34', 'reside..."
2,201902,Durch die Förderung der familienergänzenden Be...,FAVOR,Welfare,"{'gender': 'Weiblich', 'age': '35-49', 'reside..."
3,201903,Ich ziehe eine Elternzeit vor. Die Zeit nach d...,AGAINST,Welfare,"{'gender': 'Weiblich', 'age': '35-49', 'reside..."
4,201904,Unser Asylrecht muss konsequent angewendet wer...,AGAINST,Immigration,"{'gender': 'Weiblich', 'age': '35-49', 'reside..."
...,...,...,...,...,...
26330,2019028133,"È da seguire il principio ""chi inquina paga"", ...",FAVOR,Infrastructure & Environment,"{'gender': 'Weiblich', 'age': '18-34', 'reside..."
26331,2019028136,Le energie rinnovabile sono il presente e il f...,FAVOR,Infrastructure & Environment,"{'gender': 'Männlich', 'age': '18-34', 'reside..."
26332,2019028144,Attraverso un rafforzamento delle condizioni q...,FAVOR,Infrastructure & Environment,"{'gender': 'Männlich', 'age': '35-49', 'reside..."
26333,2019028149,"Molto di più di ora, dobbiamo poter installare...",FAVOR,Infrastructure & Environment,"{'gender': 'Männlich', 'age': '18-34', 'reside..."


In [5]:
keys = list(df.demographic_profile[0].keys())

In [6]:
def getValues(data, key):
    return data[key]

In [7]:
key = "gender"
df[key] = df.demographic_profile.apply(getValues, args=(key,))

In [8]:
df

,argument_id,argument,stance,topic,demographic_profile,gender
0,201900,Das Schweizer Volk hat die MEI angenommen und ...,FAVOR,Immigration,"{'gender': 'Männlich', 'age': '18-34', 'reside...",Männlich
1,201901,Eine Legalisierung von Cannabis entlasten die ...,FAVOR,Society,"{'gender': 'Männlich', 'age': '18-34', 'reside...",Männlich
2,201902,Durch die Förderung der familienergänzenden Be...,FAVOR,Welfare,"{'gender': 'Weiblich', 'age': '35-49', 'reside...",Weiblich
3,201903,Ich ziehe eine Elternzeit vor. Die Zeit nach d...,AGAINST,Welfare,"{'gender': 'Weiblich', 'age': '35-49', 'reside...",Weiblich
4,201904,Unser Asylrecht muss konsequent angewendet wer...,AGAINST,Immigration,"{'gender': 'Weiblich', 'age': '35-49', 'reside...",Weiblich
...,...,...,...,...,...,...
26330,2019028133,"È da seguire il principio ""chi inquina paga"", ...",FAVOR,Infrastructure & Environment,"{'gender': 'Weiblich', 'age': '18-34', 'reside...",Weiblich
26331,2019028136,Le energie rinnovabile sono il presente e il f...,FAVOR,Infrastructure & Environment,"{'gender': 'Männlich', 'age': '18-34', 'reside...",Männlich
26332,2019028144,Attraverso un rafforzamento delle condizioni q...,FAVOR,Infrastructure & Environment,"{'gender': 'Männlich', 'age': '35-49', 'reside...",Männlich
26333,2019028149,"Molto di più di ora, dobbiamo poter installare...",FAVOR,Infrastructure & Environment,"{'gender': 'Männlich', 'age': '18-34', 'reside...",Männlich


In [9]:
def toInt(data):
  if(data == "Weiblich"):
    return 0
  return 1

df["label"] = df.gender.apply(toInt)
df.drop(["argument_id", "stance", "topic", "demographic_profile", "gender"], axis="columns", inplace=True)

In [10]:
df

,argument,label
0,Das Schweizer Volk hat die MEI angenommen und ...,1
1,Eine Legalisierung von Cannabis entlasten die ...,1
2,Durch die Förderung der familienergänzenden Be...,0
3,Ich ziehe eine Elternzeit vor. Die Zeit nach d...,0
4,Unser Asylrecht muss konsequent angewendet wer...,0
...,...,...
26330,"È da seguire il principio ""chi inquina paga"", ...",0
26331,Le energie rinnovabile sono il presente e il f...,1
26332,Attraverso un rafforzamento delle condizioni q...,1
26333,"Molto di più di ora, dobbiamo poter installare...",1


**Convert to a Dataset Dict, split in train-validation-test**

In [11]:
data = Dataset.from_pandas(df)
train = data.train_test_split(test_size=0.25)
test_valid = train['test'].train_test_split(test_size=0.5)

# gather all to a single DatasetDict
data = DatasetDict({
    'train': train['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})

data

DatasetDict({
    train: Dataset({
        features: ['argument', 'label'],
        num_rows: 19751
    })
    test: Dataset({
        features: ['argument', 'label'],
        num_rows: 3292
    })
    valid: Dataset({
        features: ['argument', 'label'],
        num_rows: 3292
    })
})

**Tokenize:** Using padding = "longest" for quick compute due to only 15 GB GPU availability on Colab

In [12]:
modelId = "xlm-roberta-large"
tokenizer = AutoTokenizer.from_pretrained(modelId)

def tokenize(examples):
    tokenized_inputs = tokenizer(
       examples["argument"], padding="longest", truncation=True
    )
    return tokenized_inputs

tokenized_dataset = data.map(tokenize, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/19751 [00:00<?, ? examples/s]

Map:   0%|          | 0/3292 [00:00<?, ? examples/s]

Map:   0%|          | 0/3292 [00:00<?, ? examples/s]

In [13]:
labels = ["Weiblich", "Männlich"]
id2label = {0:"Weiblich", 1:"Männlich"}
label2id = {"Weiblich":0, "Männlich":1}

**Metrics**

In [14]:
import numpy as np
import evaluate

clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return clf_metrics.compute(predictions=predictions, references=labels)

**init Model and training Args**: "Longest" padding allowed a larger batch size. "Max" padding allowed a batch size of only 4

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(
    modelId,
    num_labels=2,
    id2label=id2label,
    label2id=label2id
)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="/content/sample_data",
    overwrite_output_dir = True,
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


**Train and Test**

In [17]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.670200,0.663497,0.623937,0.768425,0.623937,1.000000
2,0.661500,0.662192,0.623937,0.768425,0.623937,1.000000


TrainOutput(global_step=2470, training_loss=0.6680283519420547, metrics={'train_runtime': 3927.9977, 'train_samples_per_second': 10.057, 'train_steps_per_second': 0.629, 'total_flos': 1.057453599093906e+16, 'train_loss': 0.6680283519420547, 'epoch': 2.0})

In [18]:
trainer.evaluate()

{'eval_loss': 0.6621918678283691,
 'eval_accuracy': 0.6239368165249088,
 'eval_f1': 0.7684249906472128,
 'eval_precision': 0.6239368165249088,
 'eval_recall': 1.0,
 'eval_runtime': 85.7608,
 'eval_samples_per_second': 38.386,
 'eval_steps_per_second': 2.402,
 'epoch': 2.0}

In [19]:
trainer.evaluate(tokenized_dataset["test"])

{'eval_loss': 0.6649479866027832,
 'eval_accuracy': 0.6187727825030377,
 'eval_f1': 0.7644961531244135,
 'eval_precision': 0.6187727825030377,
 'eval_recall': 1.0,
 'eval_runtime': 89.9471,
 'eval_samples_per_second': 36.599,
 'eval_steps_per_second': 2.29,
 'epoch': 2.0}